In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Definir a arquitetura da CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(7 * 7 * 64, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.maxpool1(out)
        out = self.conv2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu3(out)
        out = self.fc2(out)
        return out

# Configurações
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 128
learning_rate = 0.001
num_epochs = 10

# Transformações nos dados
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Carregar o conjunto de dados MNIST
train_dataset = datasets.MNIST(root='data/', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='data/', train=False, transform=transform)

# Criar os dataloaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Instanciar o modelo
model = CNN().to(device)

# Definir a função de perda e o otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Treinamento do modelo
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass e otimização
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Mostrar informações do treinamento
        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')

# Avaliação do modelo
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Acurácia do modelo nos dados de teste: {accuracy:.2f}%')

# Salvar o modelo treinado
torch.save(model.state_dict(), 'mnist_cnn_model.pth')

Epoch [1/10], Step [100/469], Loss: 0.1250
Epoch [1/10], Step [200/469], Loss: 0.1284
Epoch [1/10], Step [300/469], Loss: 0.1375
Epoch [1/10], Step [400/469], Loss: 0.0123
Epoch [2/10], Step [100/469], Loss: 0.0868
Epoch [2/10], Step [200/469], Loss: 0.0906
Epoch [2/10], Step [300/469], Loss: 0.0870
Epoch [2/10], Step [400/469], Loss: 0.0278
Epoch [3/10], Step [100/469], Loss: 0.0051
Epoch [3/10], Step [200/469], Loss: 0.0116
Epoch [3/10], Step [300/469], Loss: 0.0357
Epoch [3/10], Step [400/469], Loss: 0.0414
Epoch [4/10], Step [100/469], Loss: 0.0184
Epoch [4/10], Step [200/469], Loss: 0.0410
Epoch [4/10], Step [300/469], Loss: 0.0184
Epoch [4/10], Step [400/469], Loss: 0.0751
Epoch [5/10], Step [100/469], Loss: 0.0229
Epoch [5/10], Step [200/469], Loss: 0.0056
Epoch [5/10], Step [300/469], Loss: 0.0054
Epoch [5/10], Step [400/469], Loss: 0.0148
Epoch [6/10], Step [100/469], Loss: 0.0072
Epoch [6/10], Step [200/469], Loss: 0.0084
Epoch [6/10], Step [300/469], Loss: 0.0223
Epoch [6/10